
# General information of the datascrapping process.

Here, we get vital data for the key airports we want to scrape data from:

location (coordinates, country, city), IATA code,

This then allows us, using the wikipedia API to find each airport's corresponding wikipedia link and wikipedia name.
For example JFK is encoded in wikipedia's internal database as John_F._Kennedy_International_Airport. This matches the last part of the url in the wikipedia page link for the airport:     
https://en.wikipedia.org/wiki/John_F._Kennedy_International_Airport

For each time range:

The wikipedia page's raw text can be scrapped predictiably for a list of destinations from an airport. We will encode airport destinations using iata code.

We do this for each airport, generating a large csv file of airport-destination pairs for a particular time change.

We will look at 2 time ranges (now(as of June 4th 11am Eastern Time), before Jan 1, 2000 UTC 0)

### Sources :


list of top 1000 airports by traffic to scrape:

https://gettocenter.com/airports/top-100-airports-in-world/1000#google_vignette  


detailed airport database to cross reference:

https://www.partow.net/miscellaneous/airportdatabase/index.html#Downloads 

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
# Suppress just SettingWithCopyWarning
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# Part 1: get basic data for the list of top 1000 airports 

In [2]:
# Example: load HTML from a URL or string
url = "https://gettocenter.com/airports/top-100-airports-in-world/1000"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find("table")  
rows = table.find_all("tr")

with open("./data/top_airports_basic_data.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    for row in rows:
        # Extract all cells (td or th)
        cols = row.find_all(["td", "th"])
        # Write the row text content to CSV
        writer.writerow([col.get_text(strip=True) for col in cols])
    f.close()

after replacing blank strings "", read in again csv file and add column names

In [3]:
data = pd.read_csv("./data/top_airports_basic_data.csv", names=["full_name", "iata", "city", "country", "estimated_pax"])
print(len(data))
data=data.dropna()
print(len(data))
data.head(n=11)

1087
989


,full_name,iata,city,country,estimated_pax
1.0,Hartsfield–Jackson Atlanta International Airport,ATL,Atlanta,United States,"103,902,992"
2.0,Beijing Capital International Airport,PEK,Beijing,China,"95,786,442"
3.0,Dubai International Airport,DXB,Dubai,United Arab Emirates,"88,242,099"
4.0,Los Angeles International Airport,LAX,Los Angeles,United States,"84,557,968"
5.0,O'Hare International Airport,ORD,Chicago,United States,"79,828,183"
6.0,Heathrow Airport,LHR,London,United Kingdom,"78,014,598"
7.0,Haneda Airport,HND,Tokyo,Japan,"76,476,251"
8.0,Hong Kong International Airport,HKG,Hong Kong,Hong Kong,"72,665,078"
9.0,Shanghai Pudong International Airport,PVG,Shanghai,China,"70,001,237"
10.0,Charles de Gaulle International Airport,CDG,Paris,France,"69,471,442"


we now have obtain basic information for the top airports the the world. save the pdf

In [4]:
data.to_csv("./data/top_airports_basic_data.csv")

#  Part 2:
Get wikipedia urls and for each iata code by using search api in wikipedia

In [5]:
#testing a function
def get_wikipedia_url_from_name(name):
    """Search Wikipedia using IATA code and return the best-matching article title."""
    search_url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "list": "search",
        "srsearch": f"{name}",
        "format": "json"
    }

    response = requests.get(search_url, params=params)
    data = response.json()
    try:
        raw_name = data['query']['search'][0]['title'] 
        raw_name = raw_name.replace(" ", "_") #replace the raw name spaces with _

        return "https://en.wikipedia.org/wiki/"+raw_name #format for english wiki
    except (KeyError, IndexError):
        return None
    
print(get_wikipedia_url_from_name("Hartsfield–Jackson Atlanta International Airport")) #popular airport
print(get_wikipedia_url_from_name("Gobernador Castello Airport")) #more obscure airport


https://en.wikipedia.org/wiki/Hartsfield–Jackson_Atlanta_International_Airport
https://en.wikipedia.org/wiki/Gobernador_Edgardo_Castello_Airport


now, run for all airports

In [6]:
data = pd.read_csv("./data/top_airports_basic_data.csv")

In [7]:
data["wiki_url"] =None #add column
for index, row in data.iterrows():
    try:
        print("current index:", index)
        name = row["full_name"]
        url = get_wikipedia_url_from_name(name)
        data["wiki_url"][index] = url
    except:
        data["wiki_url"][index] = None

current index: 0
current index: 1
current index: 2
current index: 3
current index: 4
current index: 5
current index: 6
current index: 7
current index: 8
current index: 9
current index: 10
current index: 11
current index: 12
current index: 13
current index: 14
current index: 15
current index: 16
current index: 17
current index: 18
current index: 19
current index: 20
current index: 21
current index: 22
current index: 23
current index: 24
current index: 25
current index: 26
current index: 27
current index: 28
current index: 29
current index: 30
current index: 31
current index: 32
current index: 33
current index: 34
current index: 35
current index: 36
current index: 37
current index: 38
current index: 39
current index: 40
current index: 41
current index: 42
current index: 43
current index: 44
current index: 45
current index: 46
current index: 47
current index: 48
current index: 49
current index: 50
current index: 51
current index: 52
current index: 53
current index: 54
current index: 55
cu

In [8]:
data.head(n=989)

,Unnamed: 0,full_name,iata,city,country,estimated_pax,wiki_url
0,1.0,Hartsfield–Jackson Atlanta International Airport,ATL,Atlanta,United States,"103,902,992",https://en.wikipedia.org/wiki/Hartsfield–Jacks...
1,2.0,Beijing Capital International Airport,PEK,Beijing,China,"95,786,442",https://en.wikipedia.org/wiki/Beijing_Capital_...
2,3.0,Dubai International Airport,DXB,Dubai,United Arab Emirates,"88,242,099",https://en.wikipedia.org/wiki/Dubai_Internatio...
3,4.0,Los Angeles International Airport,LAX,Los Angeles,United States,"84,557,968",https://en.wikipedia.org/wiki/Los_Angeles_Inte...
4,5.0,O'Hare International Airport,ORD,Chicago,United States,"79,828,183",https://en.wikipedia.org/wiki/O'Hare_Internati...
...,...,...,...,...,...,...,...
984,985.0,Noumérat - Moufdi Zakaria Airport,GHA,Ghardaia,Algeria,"45,794",https://en.wikipedia.org/wiki/Noumérat_–_Moufd...
985,986.0,Guarani International Airport,AGT,Ciudad del Este,Paraguay,"40,923",https://en.wikipedia.org/wiki/Guaraní_Internat...
986,987.0,Catarman National Airport,CRM,Catarman,Philippines,"40,237",https://en.wikipedia.org/wiki/Catarman_Nationa...
987,988.0,Sauce Viejo Airport,SFN,Santa Fe,Argentina,"37,725",https://en.wikipedia.org/wiki/Sauce_Viejo_Inte...


finding number of null entries in the wiki_url column

In [9]:
print(f"Number of null entries: {data['wiki_url'].isnull().sum()}")

Number of null entries: 7


find the null rows

In [10]:
data[data['wiki_url'].isnull()]

,Unnamed: 0,full_name,iata,city,country,estimated_pax,wiki_url
97,98.0,Liuting Airport,TAO,Qingdao,China,"23,210,530",None
98,99.0,Brisbane International Airport,BNE,Brisbane,Australia,"23,205,702",None
138,139.0,Presidente Juscelino Kubistschek International...,BSB,Brasilia,Brazil,"16,912,680",None
161,162.0,Nice-Côte d'Azur Airport,NCE,Nice,France,"13,304,782",None
671,672.0,Regional de Maringá - Sílvio Nane Junior Airport,MGF,Maringa,Brazil,"658,000",None
677,678.0,Lajes Field,TER,Lajes (terceira Island),Portugal,"631,236",None
972,973.0,Suboficial Ay Santiago Germano Airport,AFA,San Rafael,Argentina,"56,905",None


modifying the data


In [11]:
data["wiki_url"][97] = "https://en.wikipedia.org/wiki/Qingdao_Liuting_International_Airport"
data["wiki_url"][98] = "https://en.wikipedia.org/wiki/Brisbane_Airport"
data["wiki_url"][138] = "https://en.wikipedia.org/wiki/Bras%C3%ADlia_International_Airport"
data["wiki_url"][161] = "https://en.wikipedia.org/wiki/Nice_C%C3%B4te_d%27Azur_Airport"
data["wiki_url"][671] = "https://en.wikipedia.org/wiki/Maring%C3%A1_Regional_Airport"
data["wiki_url"][677] = "https://en.wikipedia.org/wiki/Lajes_Field"
data["wiki_url"][972] = "https://en.wikipedia.org/wiki/San_Rafael_Airport_(Argentina)"


saving the updated data

In [12]:
data.to_csv("./data/top_airports_basic_data.csv")

# Part 3:

Try to get wikipedia airport names with underscores to make querying easier, and testing a script to get a list of destinations from a wikipedia article

### getting wikipedia names


Populating wikipedia names in our top_airports_basic_data.csv

In [13]:
#the wikipedia name based on the url, is simply found as:
print("name:", "https://en.wikipedia.org/wiki/Victoria_International_Airport".split("https://en.wikipedia.org/wiki/")[1])

name: Victoria_International_Airport


In [14]:
data = pd.read_csv("./data/top_airports_basic_data.csv")
data.head(n=1)

,Unnamed: 0.1,Unnamed: 0,full_name,iata,city,country,estimated_pax,wiki_url
0,0,1.0,Hartsfield–Jackson Atlanta International Airport,ATL,Atlanta,United States,"103,902,992",https://en.wikipedia.org/wiki/Hartsfield–Jacks...


In [15]:
data["wiki_name"] = None 
wiki_names = []

for i, row in data.iterrows():
    url = str(row["wiki_url"])
    wiki_names.append(url.split("https://en.wikipedia.org/wiki/")[1])
    
data["wiki_name"] = wiki_names
#overwrite
data.to_csv("./data/top_airports_basic_data.csv")

# Part 4 : Transition into getting routes data (in the other jupyter notebook)

### testing script to get raw text from a wikipedia page and get a list of destination-tuples for the current destinations listed in wikipedia

In [15]:
#function to extract text between two found substrings
def extract_between(text, start, end):
    start_idx = text.find(start)
    if start_idx == -1:
        return "" #return empty string
    start_idx += len(start)
    end_idx = text.find(end, start_idx)
    if end_idx == -1:
        return ""
    return text[start_idx:end_idx]
#custom function to find the text matching the passenger destination section 
def extract_passenger_destinations(text, start, end):
    start_idx = text.find(start)
    #narrow down search text 
    if start_idx == -1:
        return "" #return empty string
    start_idx += len(start)
    start_idx += 3 #increment by 3 to account for ==
    end_idx = text.find(end, start_idx)
    if end_idx == -1:
        return ""
    return text[start_idx:end_idx]


#function to get iata code from wikipedia name
def get_iata_code(iata,ref_data=pd.read_csv("./data/top_airports_basic_data.csv")):
    matched = ref_data[ref_data["wiki_name"]==iata].iloc[0]
    return matched["iata"]
    

def get_destinations(iata_source, wiki_name):
    search_url = f"https://en.wikipedia.org/w/index.php?title={wiki_name}&action=raw"
    response = requests.get(search_url)
    full_text = response.text
    #get to the passenger destinations section marked by a predictiable substrings ()
    # Call both and choose the first non-None result
    destinations_text = extract_passenger_destinations(full_text, "Airlines and destinations","==") 
    #if the length if less than 5, a low amount, we try differently
    if("Airport" not in destinations_text): destinations_text = extract_passenger_destinations(full_text, "Airlines and destinations","Cargo") 
    destinations_list = destinations_text.split("<!-- -->")#split predictably
    
    return_list = [] #return a list of destination of tuple of the form (iata_source, starting_wiki_name, dest_wikipedia_name,airline, isSeasonal)
    #ignore the first part, since that is some disclaimer
    for i in range(1, len(destinations_list)):
        airlineDest = destinations_list[i]#we get a list relating to airlines and their list of destinations
        #get information before and the seasonal destination section (if present)
        beforeSeasonal = ""
        afterSeasonal = None
        if "'''Seasonal:'''" in airlineDest:
            seasonalSplit = airlineDest.split("'''Seasonal:'''")
            beforeSeasonal = seasonalSplit[0]
            afterSeasonal = seasonalSplit[1]
        else: 
            beforeSeasonal = airlineDest 
        
        #get the airline based on a predictable pattern (in the first element)
        beforeSeasonalsplit = beforeSeasonal.split(",") #split based on commas
        airline_firstdest = beforeSeasonalsplit[0].split(" | [[")  #split between the airline and first destination
        airline = str(airline_firstdest[0]) #extract the airline
        airline = extract_between(airline,"[[","]]") #remove the [[]]
        #do a final split just in case (even if | is not found), [0] selects the current string
        firstdest = ""  #get first destination (assuming not seasonal) 
        if len(airline_firstdest) == 2:
            destination = airline_firstdest[1].split("|")[0].strip()
            destination_wiki_name = destination.replace(" ", "_") #replace destination to get the destination iata_code
            return_list.append((iata_source,wiki_name,destination_wiki_name,airline,False )) #(iata_source, starting_wiki_name, dest_wikipedia_name,airline, isSeasonal)
        #loop thorugh the rest of the comma list for the before seasonal split if possible 
        for i in range(1, len(beforeSeasonalsplit)):
            destination = beforeSeasonalsplit[i]
            extracted_destination = extract_between(destination,"[[","]]")
            if (extracted_destination!=""): #assuming 
                destination_wiki_name = extracted_destination.split("|")[0].strip()
                destination_wiki_name = destination_wiki_name.replace(" ", "_")
                return_list.append((iata_source,wiki_name,destination_wiki_name,airline,False )) #(iata_source, starting_wiki_name, dest_wikipedia_name,airline, isSeasonal)
        #loop through the seasonal destinations list if the current airline has it
        if (afterSeasonal):
            afterSeasonalSplit = afterSeasonal.split(",")
            for i in range(0, len(afterSeasonalSplit)):
                destination = afterSeasonalSplit[i]
                extracted_destination = extract_between(destination,"[[","]]")
                if (extracted_destination!=""): #assuming 
                    destination_wiki_name = extracted_destination.split("|")[0].strip()
                    destination_wiki_name = destination_wiki_name.replace(" ", "_")
                    return_list.append((iata_source,wiki_name,destination_wiki_name,airline,True )) #(iata_source, starting_wiki_name, dest_wikipedia_name,airline, isSeasonal)
            
    return return_list


testing on various airport data

In [3]:

get_destinations("DFW","Dallas_Fort_Worth_International_Airport")

[('DFW',
  'Dallas_Fort_Worth_International_Airport',
  'Montréal–Trudeau_International_Airport',
  'Air Canada',
  True),
 ('DFW',
  'Dallas_Fort_Worth_International_Airport',
  'Montréal–Trudeau_International_Airport',
  'Air Canada Express',
  True),
 ('DFW',
  'Dallas_Fort_Worth_International_Airport',
  'Charles_de_Gaulle_Airport',
  'Air France',
  False),
 ('DFW',
  'Dallas_Fort_Worth_International_Airport',
  'Portland_International_Airport',
  'Alaska Airlines',
  False),
 ('DFW',
  'Dallas_Fort_Worth_International_Airport',
  'Seattle–Tacoma_International_Airport',
  'Alaska Airlines',
  False),
 ('DFW',
  'Dallas_Fort_Worth_International_Airport',
  'Albany_International_Airport',
  'American Airlines',
  False),
 ('DFW',
  'Dallas_Fort_Worth_International_Airport',
  'Albuquerque_International_Sunport',
  'American Airlines',
  False),
 ('DFW',
  'Dallas_Fort_Worth_International_Airport',
  'Hartsfield–Jackson_Atlanta_International_Airport',
  'American Airlines',
  False),

In [4]:

get_destinations("JFK","John_F._Kennedy_International_Airport")

[('JFK',
  'John_F._Kennedy_International_Airport',
  'Dublin_Airport',
  'Aer Lingus',
  False),
 ('JFK',
  'John_F._Kennedy_International_Airport',
  'Manchester_Airport',
  'Aer Lingus',
  False),
 ('JFK',
  'John_F._Kennedy_International_Airport',
  'Shannon_Airport',
  'Aer Lingus',
  False),
 ('JFK',
  'John_F._Kennedy_International_Airport',
  'Mexico_City_International_Airport',
  'Aeroméxico',
  False),
 ('JFK',
  'John_F._Kennedy_International_Airport',
  'Monterrey_International_Airport',
  'Aeroméxico',
  True),
 ('JFK',
  'John_F._Kennedy_International_Airport',
  'Montréal–Trudeau_International_Airport',
  'Air Canada Express',
  False),
 ('JFK',
  'John_F._Kennedy_International_Airport',
  'Toronto_Pearson_International_Airport',
  'Air Canada Express',
  False),
 ('JFK',
  'John_F._Kennedy_International_Airport',
  'Beijing_Capital_International_Airport',
  'Air China',
  False),
 ('JFK',
  'John_F._Kennedy_International_Airport',
  'Madrid–Barajas_Airport',
  'Air Euro

In [5]:
get_destinations("PEK","Beijing_Capital_International_Airport")

[('PEK',
  'Beijing_Capital_International_Airport',
  'Houari_Boumediene_Airport',
  'Air Algérie',
  False),
 ('PEK',
  'Beijing_Capital_International_Airport',
  'Almaty_International_Airport',
  'Air Astana',
  False),
 ('PEK',
  'Beijing_Capital_International_Airport',
  'Vancouver_International_Airport',
  'Air Canada',
  False),
 ('PEK',
  'Beijing_Capital_International_Airport',
  'Aksu_Hongqipo_Airport',
  'Air China',
  False),
 ('PEK',
  'Beijing_Capital_International_Airport',
  'Nursultan_Nazarbayev_International_Airport',
  'Air China',
  False),
 ('PEK',
  'Beijing_Capital_International_Airport',
  'Athens_International_Airport',
  'Air China',
  False),
 ('PEK',
  'Beijing_Capital_International_Airport',
  'Auckland_Airport',
  'Air China',
  False),
 ('PEK',
  'Beijing_Capital_International_Airport',
  'Suvarnabhumi_Airport',
  'Air China',
  False),
 ('PEK',
  'Beijing_Capital_International_Airport',
  'Josep_Tarradellas_Barcelona–El_Prat_Airport',
  'Air China',
  Fal

In [6]:
get_destinations("PAP","Toussaint_Louverture_International_Airport")

[('PAP',
  'Toussaint_Louverture_International_Airport',
  'Orly_Airport',
  'Air Caraïbes',
  False),
 ('PAP',
  'Toussaint_Louverture_International_Airport',
  'Pointe-à-Pitre_International_Airport',
  'Air France',
  False),
 ('PAP',
  'Toussaint_Louverture_International_Airport',
  'Montréal-Pierre_Elliott_Trudeau_International_Airport',
  'Air Transat',
  False),
 ('PAP',
  'Toussaint_Louverture_International_Airport',
  'Miami_International_Airport',
  'American Airlines',
  False),
 ('PAP',
  'Toussaint_Louverture_International_Airport',
  'Providenciales_International_Airport',
  'Caicos Express Airways',
  False),
 ('PAP',
  'Toussaint_Louverture_International_Airport',
  'Providenciales_International_Airport',
  'InterCaribbean Airways',
  False),
 ('PAP',
  'Toussaint_Louverture_International_Airport',
  'Fort_Lauderdale–Hollywood_International_Airport',
  'JetBlue',
  False),
 ('PAP',
  'Toussaint_Louverture_International_Airport',
  'John_F._Kennedy_International_Airport',

In [ ]:
get_destinations("JFK","John_F._Kennedy_International_Airport")

In [ ]:
get_destinations("VDM","Gobernador_Edgardo_Castello_Airport")



[]

In [10]:
get_destinations("FRA","Frankfurt_Airport")


[('FRA',
  'Frankfurt_Airport',
  'Athens_International_Airport',
  'Aegean Airlines',
  False),
 ('FRA',
  'Frankfurt_Airport',
  'Thessaloniki_International_Airport',
  'Aegean Airlines',
  False),
 ('FRA',
  'Frankfurt_Airport',
  'Heraklion_International_Airport',
  'Aegean Airlines',
  True),
 ('FRA', 'Frankfurt_Airport', 'Dublin_Airport', 'Aer Lingus', False),
 ('FRA',
  'Frankfurt_Airport',
  'Houari_Boumediene_Airport',
  'Air Algérie',
  False),
 ('FRA',
  'Frankfurt_Airport',
  'Almaty_International_Airport',
  'Air Astana',
  False),
 ('FRA',
  'Frankfurt_Airport',
  'Nursultan_Nazarbayev_International_Airport',
  'Air Astana',
  False),
 ('FRA', 'Frankfurt_Airport', 'Oral_Ak_Zhol_Airport', 'Air Astana', False),
 ('FRA',
  'Frankfurt_Airport',
  'Hurghada_International_Airport',
  'Air Cairo',
  False),
 ('FRA',
  'Frankfurt_Airport',
  'Marsa_Alam_International_Airport',
  'Air Cairo',
  False),
 ('FRA',
  'Frankfurt_Airport',
  'Sharm_El_Sheikh_International_Airport',
  'A

In [11]:

get_destinations("CAN", "Guangzhou_Baiyun_International_Airport")

[]

In [7]:
get_destinations("OUI","Ushant_Airport")

[('OUI', 'Ushant_Airport', 'Brest_Bretagne_Airport', "Finist'air", False)]